In [5]:
import sys, importlib
import ncbi_crassvirales as ncbi_crassvirales
import utils as utils
from ete3 import RectFace, faces, Tree
import itertools

/home/dani/miniconda3/lib/python3.9/site-packages/ete3-3.1.2-py3.7.egg/ete3/evol/parser/codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?
/home/dani/miniconda3/lib/python3.9/site-packages/ete3-3.1.2-py3.7.egg/ete3/evol/parser/codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?


In [7]:
#summaries_dir = "/home/danielc/projects/Bas_phages/5_nr_screening/2_hits_summary/"
summaries_dir = "/home/dani/encode_home//projects/Bas_phages/5_nr_screening/2_hits_summary/"


#trees_dir = "/home/danielc/projects/Bas_phages/5_nr_screening/4_merged_ncbi_crassvirales/2_trees"
trees_dir = "/home/dani/encode_home/projects/Bas_phages/5_nr_screening/4_merged_ncbi_crassvirales/2_trees"



vs2_dir = "/home/danielc/projects/Bas_phages/5_nr_screening/4_merged_ncbi_crassvirales/3_bacterial/old"
vs2_dir = "/home/dani/encode_home/projects/Bas_phages/5_nr_screening/4_merged_ncbi_crassvirales/3_bacterial/old"


In [8]:
# get interesting CLs to look at
#importlib.reload(ncbi_crassvirales);
#ncbi_crassvirales.get_interesting_cls(trees_dir, summaries_dir)

In [15]:
cl_id = "cl_s_084" #
cl_id = "cl_2122" # 
cl_id = "cl_s_430" # 
cl_id = "cl_s_543" #
cl_id = "cl_2336" ##
cl_id = "cl_0453"
cl_id = "cl_2097" ##
cl_id = "cl_s_720" # show
cl_id = "cl_5837"
cl_id = "cl_s_457" # 
cl_id = "cl_s_032" ##
cl_id = "cl_0227" ##
cl_id = "cl_0132"
cl_id = "cl_0144"
cl_id = "cl_0163"
cl_id = "cl_0450"
cl_id = "cl_0473" #####
cl_id = "cl_0786"
cl_id = "cl_0768"
cl_id = "cl_0671"
cl_id = "cl_s_702" ###
#cl_id = "cl_s_704"
cl_id = "cl_1764"
cl_id = "cl_s_422"
cl_id = "cl_0144"
cl_id = "cl_s_084"
cl_id = "cl_0633"
cl_id = "cl_s_121"
cl_id = "cl_1092"
#cl_id = "cl_0914"
cl_id = "cl_s_330"
cl_id = "cl_2674"
cl_id = "cl_0467" 
cl_id = "cl_0193"
cl_id = "cl_1607"
cl_id = "cl_3257"
cl_id = "cl_0815"
cl_id = "cl_2636"
cl_id = "cl_1952" #
cl_id = "cl_6497"
cl_id = "cl_0008"
cl_id = "cl_s_010" ##
#cl_id = "cl_1011"
cl_id = "cl_1459"
cl_id = "cl_1237"
cl_id = "cl_s_350"

In [16]:
# parse ncbi summary table
importlib.reload(ncbi_crassvirales);
ncbi_summary_df = ncbi_crassvirales.parse_ncbi_summary_table(cl_id, summaries_dir)
#ncbi_summary_df.loc["HDS29336.1",]

/tmp/ipykernel_9656/712990510.py:3: DtypeWarning: Columns (9,10,12,13,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  ncbi_summary_df = ncbi_crassvirales.parse_ncbi_summary_table(cl_id, summaries_dir)


In [17]:
# parse crassvirales taxonomy 
importlib.reload(utils);

crassvirales_taxonomy_file = "crassphages_taxonomy_terL_and_new.txt"
crassvirales_taxonomy_df = utils.get_taxonomy_df(crassvirales_taxonomy_file)

In [18]:
# read prophage annotation
#importlib.reload(ncbi_crassvirales);

#annot = ncbi_crassvirales.parse_vs2_annots(vs2_dir)

In [19]:
#annot

In [20]:
# parse and annotate the ncbi & Crassvirales tree
importlib.reload(ncbi_crassvirales);

t = ncbi_crassvirales.read_and_annotate_tree(cl_id, trees_dir, ncbi_summary_df, crassvirales_taxonomy_df)

#t = ncbi_crassvirales.add_phylum_percent_inner_nodes(t)
#t = ncbi_crassvirales.collapse_nodes_phylum_perc(t, 0.9)
t = ncbi_crassvirales.color_format_tree(t)
t.show()


/home/dani/encode_home/projects/Bas_phages/5_nr_screening/4_merged_ncbi_crassvirales/2_trees/cl_s_350_ncbi_trimmed.nw


In [173]:
def create_target_taxa_groups():
    taxa_groups_raw = ["crassvirales","NA", "new", "Viruses", "bact_crass_polytomy", "prophage"]
    taxa_groups = list()
    # create the target taxa_groups to look for in the tree
    for i in range(2, len(taxa_groups_raw)+1):
        combinations = itertools.combinations(taxa_groups_raw, i)
        # iterate the combinations and retain those with "crassvirales" in them
        for combination in combinations:
            if "crassvirales" in combination:
                taxa_groups.append(list(combination))

    return taxa_groups

taxa_groups = create_target_taxa_groups()
taxa_groups = [["crassvirales"]] + taxa_groups

In [174]:
def resolve_crass_bacteria_polytomies(t):
    '''
    Looks for clades with Crassvirales and Bacteria, all of them with 0 dist
    '''
    checked_leaves = list()
    crass_polytom_leaves = t.search_nodes(dist=0.0, superkingdom="crassvirales")
    for leaf in crass_polytom_leaves:
        if leaf.name not in checked_leaves:
            checked_leaves.append(leaf.name)
            # go one node up, check superkingdom of descendants
            node_up = leaf.up
            for node in node_up.iter_descendants():
                if node.superkingdom == "Bacteria":
                    node.superkingdom = "bact_crass_polytomy"
                    print(f"\t- polytomy bacteria-crass, {node.name}")

    return t

t = resolve_crass_bacteria_polytomies(t)

In [155]:
def identify_crassvirales_mrcas(t, cl_id, taxa_groups, start_cont, faces=False):
    '''

    '''

    monophyletic_clades_leaves = dict()
    monophyletic_clades = dict()
    mrcas_ids_to_clean = list()
    cont = 0
    for taxa_group in taxa_groups:
        mrcas = t.get_monophyletic(target_attr="superkingdom", values=taxa_group)
        if mrcas:
            for mrca in mrcas:
                cont += 1
                mrca_id = f"mrca_{cont}"
                # get names of the leaves
                mrca_leaves_names = {leaf.name for leaf in mrca.iter_leaves()}
                if monophyletic_clades_leaves:
                    for target_mrca, target_leaves_names in monophyletic_clades_leaves.items():
                        tmp = target_leaves_names.union(mrca_leaves_names)
                        if len(tmp) == len(mrca_leaves_names):
                            mrcas_ids_to_clean.append(target_mrca)

                monophyletic_clades_leaves[mrca_id] = mrca_leaves_names
                monophyletic_clades[mrca_id] = mrca


    # remove MRCAs already contained in higher MRCAs
    mrcas_ids_to_clean = list(set(mrcas_ids_to_clean))
    for mrca_id in mrcas_ids_to_clean:
        del monophyletic_clades[mrca_id]
        del monophyletic_clades_leaves[mrca_id]


    # change mrca_ids to lower numbers
    final_monophyletic_clades = dict()
    final_cont = start_cont
    for mrca_id, mrca in monophyletic_clades.items():
        final_monophyletic_clades[f"mrca_{final_cont}"] = mrca
        if faces:
            mrca.add_features(mrca_id=f"mrca_{final_cont}")
            face = RectFace(5,5, "grey","grey", label=f"mrca_{final_cont}")
            mrca.add_face(face, 0, "float")
        final_cont +=1

    #if faces:
    #    pdf_outfile = f"/home/danielc/projects/Bas_phages/5_nr_screening/4_merged_ncbi_crassvirales/4_MRCAs_taxa/{cl_id}.pdf"
    #    t.render(pdf_outfile)

    return t, final_monophyletic_clades, final_cont

start_cont = 1
t, mrcas, cont = identify_crassvirales_mrcas(t, cl_id, taxa_groups, start_cont, faces=True)

In [141]:
print(mrcas)
t.show()

{'mrca_1': Tree node '0.694' (0x7fc97ae528e), 'mrca_2': Tree node '0.172' (0x7fc967a6549)}



--MBQ7819935.1
MBQ7819935.1

--WP_194270469.1
WP_194270469.1


In [168]:
for node_name, tree in rooted_trees.items():
    print(node_name, tree.name)
    tree.show()

MBQ7819935.1 
WP_194270469.1 


In [ ]:
taxa_groups = [["crassvirales","NA", "new", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "new", "NA", "Viruses"],
               ["crassvirales", "new", "NA", "bact_crass_polytomy"],
               ["crassvirales", "new", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "NA", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "new", "NA"],
               ["crassvirales", "new", "Viruses"],
               ["crassvirales", "new", "bact_crass_polytomy"],
               ["crassvirales", "NA", "Viruses"],
               ["crassvirales", "NA", "bact_crass_polytomy"],
               ["crassvirales", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "new"],
               ["crassvirales", "NA"],
               ["crassvirales", "Viruses"],
               ["crassvirales", "bact_crass_polytomy"],
               ["crassvirales"]]
               
monophyletic_clades_leaves = dict()
monophyletic_clades = dict()
mrcas_ids_to_clean = list()
cont = 0
for taxa_group in reversed(taxa_groups):
    mrcas = t.get_monophyletic(target_attr="superkingdom", values=taxa_group)
    if mrcas:
        for mrca in mrcas:
            cont += 1
            mrca_id = f"mrca_{cont}"
            # get names of the leaves
            mrca_leaves_names = {leaf.name for leaf in mrca.iter_leaves()}
            if monophyletic_clades_leaves:
                for target_mrca, target_leaves_names in monophyletic_clades_leaves.items():
                    tmp = target_leaves_names.union(mrca_leaves_names)
                    if len(tmp) == len(mrca_leaves_names):
                        mrcas_ids_to_clean.append(target_mrca)
                    
            monophyletic_clades_leaves[mrca_id] = mrca_leaves_names
            monophyletic_clades[mrca_id] = mrca
                        
mrcas_ids_to_clean   = list(set(mrcas_ids_to_clean))
for mrca_id in mrcas_ids_to_clean:
    del monophyletic_clades[mrca_id]
    del monophyletic_clades_leaves[mrca_id]


for mrca_id, mrca in monophyletic_clades.items():
    face = RectFace(5,5, "red","red", label=mrca_id)
    mrca.add_face(face, 0, "float")
print(monophyletic_clades)

{'mrca_15': Tree node '0.694' (0x7fc97ae528e), 'mrca_16': Tree node '0.172' (0x7fc967a6549)}


In [ ]:
for mrca_id, mrca in monophyletic_clades.items():
    if mrca.up.is_root():
        print("lol")

In [36]:
t.show()

In [ ]:
        
# get the most distant node to the mrca
rooted_trees = dict()
for mrca_id, mrca in monophyletic_clades.items():
    bacteria_leaves_dists = {bacteria_leaf:mrca.get_distance(bacteria_leaf) for bacteria_leaf in t.search_nodes(superkingdom="Bacteria")}
    # sort by distance
    bacteria_leaves_dists = {node: distance for node, distance in sorted(bacteria_leaves_dists.items(), key=lambda item: item[1], reverse=True)}
    farthest_bact_leaf = list(bacteria_leaves_dists.keys())[0]
    # root using the farthest Bacteria
    t.set_outgroup(farthest_bact_leaf)
    #print(bacteria_leaves_dists)
    print(mrca_id, farthest_bact_leaf.name)
    #t.show()
    rooted_trees[farthest_bact_leaf.name] = t


mrca_15 OGG60168.1
mrca_16 MBQ1387579.1


In [30]:
rooted_trees

{'NCX93431.1': Tree node '' (0x7fe8d672d6d)}

In [ ]:
t.show()

In [50]:
taxa_groups = [["crassvirales","NA", "new", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "new", "NA", "Viruses"],
               ["crassvirales", "new", "NA", "bact_crass_polytomy"],
               ["crassvirales", "new", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "NA", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "new", "NA"],
               ["crassvirales", "new", "Viruses"],
               ["crassvirales", "new", "bact_crass_polytomy"],
               ["crassvirales", "NA", "Viruses"],
               ["crassvirales", "NA", "bact_crass_polytomy"],
               ["crassvirales", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "new"],
               ["crassvirales", "NA"],
               ["crassvirales", "Viruses"],
               ["crassvirales", "bact_crass_polytomy"],
               ["crassvirales"]]
               
monophyletic_clades_leaves = dict()
monophyletic_clades = dict()
mrcas_ids_to_clean = list()
cont = 0
for taxa_group in reversed(taxa_groups):
    mrcas = t.get_monophyletic(target_attr="superkingdom", values=taxa_group)
    if mrcas:
        for mrca in mrcas:
            cont += 1
            mrca_id = f"mrca_{cont}"
            # get names of the leaves
            mrca_leaves_names = {leaf.name for leaf in mrca.iter_leaves()}
            if monophyletic_clades_leaves:
                for target_mrca, target_leaves_names in monophyletic_clades_leaves.items():
                    tmp = target_leaves_names.union(mrca_leaves_names)
                    if len(tmp) == len(mrca_leaves_names):
                        mrcas_ids_to_clean.append(target_mrca)
                    
            monophyletic_clades_leaves[mrca_id] = mrca_leaves_names
            monophyletic_clades[mrca_id] = mrca
                        
mrcas_ids_to_clean   = list(set(mrcas_ids_to_clean))
for mrca_id in mrcas_ids_to_clean:
    del monophyletic_clades[mrca_id]
    del monophyletic_clades_leaves[mrca_id]


for mrca_id, mrca in monophyletic_clades.items():
    face = RectFace(5,5, "blue","blue", label=mrca_id)
    mrca.add_face(face, 0, "float")
print(monophyletic_clades)

{'mrca_27': Tree node '0.891' (0x7fcb51725ca), 'mrca_34': Tree node '0.752' (0x7fcb48344b2)}


In [51]:
t.show()

In [24]:
mrca.get_sisters()

[Tree node 'MBQ0113515.1' (0x7f89acf445b)]

In [156]:
t = Tree("/home/dani/encode_home/OG_599_trimmed.nw", format=1)

In [157]:
t.show()